# 0 - Notebook set up 

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 34.0 MB/s 
     |████████████████████████████████| 895 kB 60.3 MB/s 
     |████████████████████████████████| 6.5 MB 63.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 61.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 10.2 MB/s 


In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 8.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=fbd75bd6f5b8f42e97f5e549fb9f5de0c4439615b82ee3be2b67d1555c67a2d0
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import json

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, OrdinalEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb

import keras
import torch
import tensorflow
from keras.layers import Dense, Flatten, Activation, Conv2D, MaxPooling2D,Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import EarlyStopping

import transformers
import sentencepiece
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, XLNetTokenizer, XLNetModel, AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

datapath = '/content/gdrive/MyDrive/data/'
split = 'train'
savepath = 'res/answer/'
index = 'VIX_1w'

with open(f'{datapath}/{split}/{index}.json', 'r') as fp:
    data = json.load(fp)
data_df = pd.read_json(f'{datapath}/{split}/{index}.json')
data_df.head(5)

,speech,stock,target_classif,target_reg
0,"[{'ECB': ['"" Good morning. I very much appreci...","[-0.472958207130432, -0.172407612204551, -0.19...",0,-0.408630
1,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.45292150974273604, -0.42023003101348805, -...",0,-0.585796
2,"[{'ECB': [], 'FED': []}, {'ECB': [' “A curren...","[-0.591069281101226, -0.36011990904808, -0.489...",1,0.469822
3,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.6796526312828061, -0.7492538094520561, -0....",0,0.127088
4,[{'ECB': [' How to preserve and consolidate t...,"[-0.021605037152767, -0.056405633687973, -0.05...",1,0.441348


# 1 - Dataset pre-processing

Based on the assumptions we detail in the full report, we extract the last speech of each central bank for each sample (line)

In [ ]:
data_df['ECB_speech']=data_df.apply(lambda x: [], axis=1)
data_df['FED_speech']=data_df.apply(lambda x: [], axis=1)
data_df['last_ECB_speech']=data_df.apply(lambda x: [], axis=1)
data_df['last_FED_speech']=data_df.apply(lambda x: [], axis=1)
data_df['last_ECB_speech_sum'] = ''
data_df['last_FED_speech_sum'] = ''

for i in range(data_df.shape[0]):
    sample_speeches = data_df['speech'][i]
    ecb_speech_list=[]
    for day in sample_speeches:
        data_df['ECB_speech'][i].append(day['ECB'])
        data_df['FED_speech'][i].append(day['FED'])

    c_ecb = -1
    c_fed = -1

    if data_df['ECB_speech'][i] != [[], [], [], [], [], [], [], [], [], [],[],[], [], [], [], [], [], [], [], []]:
      while len(data_df['last_ECB_speech'][i])<1:
          data_df['last_ECB_speech'][i] = data_df['ECB_speech'][i][c_ecb]
          c_ecb = c_ecb - 1
    else:
        data_df['last_ECB_speech'][i] =[]

    if data_df['FED_speech'][i] != [[], [], [], [], [], [], [], [], [], [],[],[], [], [], [], [], [], [], [], []]:
        while len(data_df['last_FED_speech'][i])<1:
            data_df['last_FED_speech'][i] = data_df['FED_speech'][i][c_fed]
            c_fed = c_fed - 1
    else:
        data_df['last_FED_speech'][i] =[]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_df['last_ECB_speech_clean'] = ''
data_df['last_FED_speech_clean'] = ''

for i in range(data_df.shape[0]):
        if data_df['last_FED_speech'][i] != []:
            data_df['last_FED_speech_clean'][i] = data_df['last_FED_speech'][i][-1]
        else:
            data_df['last_FED_speech_clean'][i] =''
        
        if data_df['last_ECB_speech'][i] != []:
            data_df['last_ECB_speech_clean'][i] = data_df['last_ECB_speech'][i][-1]
        else:
            data_df['last_ECB_speech_clean'][i] =''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_df

,speech,stock,target_classif,target_reg,ECB_speech,FED_speech,last_ECB_speech,last_FED_speech,last_ECB_speech_sum,last_FED_speech_sum,last_ECB_speech_clean,last_FED_speech_clean
0,"[{'ECB': ['"" Good morning. I very much appreci...","[-0.472958207130432, -0.172407612204551, -0.19...",0,-0.408630,"[["" Good morning. I very much appreciate your ...","[[], [ How will historians judge our handling...",[ Completing Economic and Monetary Union Sp...,[ Press briefing on new monetary and financia...,,,Completing Economic and Monetary Union Spe...,Press briefing on new monetary and financial...
1,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.45292150974273604, -0.42023003101348805, -...",0,-0.585796,"[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...","[""When we convened in Jackson Hole in August 2...",[],,,"""When we convened in Jackson Hole in August 20...",
2,"[{'ECB': [], 'FED': []}, {'ECB': [' “A curren...","[-0.591069281101226, -0.36011990904808, -0.489...",1,0.469822,"[[], [ “A currency beyond the nation state: ...","[[], [ “Monetary policy and the euro area pro...",[ Global and domestic inflation Speech by M...,[ Auf neuen Wegen zum alten Ziel Rede von Y...,,,Global and domestic inflation Speech by Ma...,Auf neuen Wegen zum alten Ziel Rede von Yv...
3,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.6796526312828061, -0.7492538094520561, -0....",0,0.127088,"[[], [], [], [ Assessing the new phase of unc...","[[], [], [], [], [], [], [ Understanding Infl...",[ The low interest rate environment in the eu...,[ Medium to long-term priorities of Capital M...,,,The low interest rate environment in the eur...,Medium to long-term priorities of Capital Ma...
4,[{'ECB': [' How to preserve and consolidate t...,"[-0.021605037152767, -0.056405633687973, -0.05...",1,0.441348,[[ How to preserve and consolidate the remark...,"[[], [], [], [], [], [], [], [], [], [], [], [...","[ It seems to me, paradoxically, that both lon...",[ Reflections on the current financial market...,,,"It seems to me, paradoxically, that both long...",Reflections on the current financial markets...
...,...,...,...,...,...,...,...,...,...,...,...,...
1249,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.005786586087197, -0.09542448073625501, -0....",0,-0.124952,"[[], [], ["" Good morning. It's my pleasure to ...","[[], [], [ Household Finance and Consumption ...",[ Bedroht die Schuldenkrise den Euro? Rede ...,[ Household Finance and Consumption - Opening...,,,Bedroht die Schuldenkrise den Euro? Rede v...,Household Finance and Consumption - Opening ...
1250,"[{'ECB': [], 'FED': []}, {'ECB': [' Speech on...","[-0.167134791612625, 0.11337908357381801, 0.08...",0,-0.256773,"[[], [ Speech on the occasion of the inaugura...","[[], [], [ Whither Europe after the crisis? ...",[ Intervento alla presentazione del libro di ...,[ La surveillance de la compétitivité dans un...,,,Intervento alla presentazione del libro di F...,La surveillance de la compétitivité dans une...
1251,"[{'ECB': [], 'FED': []}, {'ECB': ['"" In previo...","[1.474820494651794, 1.799626111984253, 1.41787...",0,1.379910,"[[], ["" In previous speeches I have called on ...","[[], [ Finanz- und Schuldenkrise: Wo stehen w...",[ The Triffin dilemma revisited Speech by L...,[ Preventing spillovers on the global economy...,,,The Triffin dilemma revisited Speech by Lo...,Preventing spillovers on the global economy ...
1252,[{'ECB': [' The recent sharp increases in subp...,"[-0.709180414676666, -0.7439810037612911, -0.6...",1,-0.344301,[[ The recent sharp increases in subprime mort...,"[[], [], [], [], [], [ Introductory remarks a...",[ Thank you for the opportunity to speak toda...,[ International Monetary Conference Central B...,,,Thank you for the opportunity to speak today...,International Monetary Conference Central Ba...


# 2 - Feature engineering

### 2-1 Summarizing last speech

In [ ]:
modelname = "human-centered-summarization/financial-summarization-pegasus"
model = AutoModelForSeq2SeqLM.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = model.to(device)

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

In [ ]:
def summarize_text(text):
  with torch.no_grad():
    torch.cuda.empty_cache()
    batched_sentences = []
    text_split_sent = str(text).split('.')
    
    # We split the speech by sentences and initialise our batched_sentences list that will make groups of 4 sentences
    
    for i in range((len(text_split_sent)//4)+1):
        j = i*4
        indexes = text_split_sent[j:j+4]
        separator ='.'
        pasted_sentences = separator.join(indexes)
        batched_sentences.append(pasted_sentences)
        
    # Now our batched sentences are ready, we can summarize them
    # In order to speed up the computations we use cuda (GPU), however the VRAM doesn't support the summarization of more than 8 batches = 32 sentences simultaneously

    nb_of_batches_for_cuda = len(batched_sentences)//8
    text_summarized =''

    for m in range(max(nb_of_batches_for_cuda,1)):

        n = m*8

        input_ids = tokenizer(batched_sentences[n:min(n+8,len(batched_sentences))], return_tensors="pt", padding=True).to(device).input_ids
        output = model.generate(
            input_ids, 
            max_length=512, 
            num_beams=3, 
            early_stopping=True)

        for k in range(output.shape[0]):
            decoded_batch = tokenizer.decode(output[k], skip_special_tokens=True)
            text_summarized = text_summarized + ' ' + decoded_batch

        torch.cuda.empty_cache()

    return(text_summarized)

In [ ]:
for i in range(data_df.shape[0]):
  data_df['last_ECB_speech_sum'] = summarize_text(data_df['last_ECB_speech_clean'][i])
  data_df['last_FED_speech_sum'] = summarize_text(data_df['last_FED_speech_clean'][i])

### 2-2 Sentiment analysis and question answering

We will compute 4 different features : 
- 2 sentiment analyses provided by finance-finetuned transformers
- an emotion classifier 
- a question-answer metric

In [ ]:
sentiment_1 = pipeline('sentiment-analysis', model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
sentiment_2 = pipeline('sentiment-analysis', model="ProsusAI/finbert")
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
pipeline_qa = pipeline(model='deepset/roberta-base-squad2', tokenizer='deepset/roberta-base-squad2', revision="v1.0", task="question-answering")

Downloading:   0%|          | 0.00/933 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/189 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [ ]:
data_df['sentiment_1_ecb'] = ''
data_df['sentiment_2_ecb'] = ''
data_df['emotion_qualifier_ecb'] = ''
data_df['qa_ecb'] = ''

data_df['sentiment_1_fed'] = ''
data_df['sentiment_2_fed'] = ''
data_df['emotion_qualifier_fed'] = ''
data_df['qa_fed'] = ''

In [ ]:
data_df.head()

,speech,stock,target_classif,target_reg,ECB_speech,FED_speech,last_ECB_speech,last_FED_speech,last_ECB_speech_sum,last_FED_speech_sum,last_ECB_speech_clean,last_FED_speech_clean,sentiment_1_ecb,sentiment_2_ecb,emotion_qualifier_ecb,qa_ecb,sentiment_1_fed,sentiment_2_fed,emotion_qualifier_fed,qa_fed
0,"[{'ECB': ['"" Good morning. I very much appreci...","[-0.472958207130432, -0.172407612204551, -0.19...",0,-0.408630,"[["" Good morning. I very much appreciate your ...","[[], [ How will historians judge our handling...",[ Completing Economic and Monetary Union Sp...,[ Press briefing on new monetary and financia...,Jean-Claude Trichet gives speech at State of ...,Two new and two extended sets of statistics t...,Completing Economic and Monetary Union Spe...,Press briefing on new monetary and financial...,,,,,,,,
1,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.45292150974273604, -0.42023003101348805, -...",0,-0.585796,"[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [...","[""When we convened in Jackson Hole in August 2...",[],Jean-Claude Trichet gives speech at State of ...,Two new and two extended sets of statistics t...,"""When we convened in Jackson Hole in August 20...",,,,,,,,,
2,"[{'ECB': [], 'FED': []}, {'ECB': [' “A curren...","[-0.591069281101226, -0.36011990904808, -0.489...",1,0.469822,"[[], [ “A currency beyond the nation state: ...","[[], [ “Monetary policy and the euro area pro...",[ Global and domestic inflation Speech by M...,[ Auf neuen Wegen zum alten Ziel Rede von Y...,Jean-Claude Trichet gives speech at State of ...,Two new and two extended sets of statistics t...,Global and domestic inflation Speech by Ma...,Auf neuen Wegen zum alten Ziel Rede von Yv...,,,,,,,,
3,"[{'ECB': [], 'FED': []}, {'ECB': [], 'FED': []...","[-0.6796526312828061, -0.7492538094520561, -0....",0,0.127088,"[[], [], [], [ Assessing the new phase of unc...","[[], [], [], [], [], [], [ Understanding Infl...",[ The low interest rate environment in the eu...,[ Medium to long-term priorities of Capital M...,Jean-Claude Trichet gives speech at State of ...,Two new and two extended sets of statistics t...,The low interest rate environment in the eur...,Medium to long-term priorities of Capital Ma...,,,,,,,,
4,[{'ECB': [' How to preserve and consolidate t...,"[-0.021605037152767, -0.056405633687973, -0.05...",1,0.441348,[[ How to preserve and consolidate the remark...,"[[], [], [], [], [], [], [], [], [], [], [], [...","[ It seems to me, paradoxically, that both lon...",[ Reflections on the current financial market...,Jean-Claude Trichet gives speech at State of ...,Two new and two extended sets of statistics t...,"It seems to me, paradoxically, that both long...",Reflections on the current financial markets...,,,,,,,,


In [ ]:
for i in range(data_df.shape[0]):
    
    text_in_fed = str(data_df['last_FED_speech_sum'][i])
    text_in_ecb = str(data_df['last_ECB_speech_sum'][i])
    
    ## Filling ECB data
    
    data_df['sentiment_1_ecb'][i] = sentiment_1(text_in_ecb, max_length=512, truncation=True)
    data_df['sentiment_2_ecb'][i] = sentiment_2(text_in_ecb, max_length=512, truncation=True)
    data_df['emotion_qualifier_ecb'][i] = emotion_classifier(text_in_ecb, max_length=512, truncation=True)
    
    QA_input_ecb = {
    'question': 'Is market volatile?',
    'context': text_in_ecb
}
    data_df['qa_ecb'][i] = pipeline_qa(QA_input_ecb)
    
    ## Filling FED data
    
    data_df['sentiment_1_fed'][i] = sentiment_1(text_in_fed, max_length=512, truncation=True)
    data_df['sentiment_2_fed'][i] = sentiment_2(text_in_fed, max_length=512, truncation=True)
    data_df['emotion_qualifier_fed'][i] = emotion_classifier(text_in_fed, max_length=512, truncation=True)
    
    QA_input_fed = {
    'question': 'Is market volatile?',
    'context': text_in_fed
}
    data_df['qa_fed'][i] = pipeline_qa(QA_input_fed)